In [6]:
from openfermion.ops import QubitOperator
import numpy as np
import cirq

# Maths for measuring PauliWord

1. First we create the ansatz wavefunction (via ansatz circuit):

$$ U_{UCCSD}|HF\rangle = |\psi_{UCCSD}\rangle$$ 

2. Overall goal is to measure the energy:

$$ E =  \langle  H \rangle =  \langle \psi_{UCCSD}| H |\psi_{UCCSD}\rangle$$ 

- the Hamiltonian is given as a summation of PauliWords:

$$H = \sum_{i} \alpha_{i} P_{i}$$ 

- where each term is measured as:

$$ E_{i} =  \langle  P_{i} \rangle =  \langle \psi_{UCCSD}| P_{i} |\psi_{UCCSD}\rangle$$ 

- and:

$$E = \sum_{i} E_{i}$$ 


To do this via a **quantum circuit** we do the following:

$$ E_{i} =  \langle \psi_{UCCSD}| P_{i} |\psi_{UCCSD}\rangle = M_{z} U_{i}|\psi_{UCCSD}\rangle$$ 

where:
- $ U_{i}$ is a change of basis of $P_{i}$ to Z basis for measurement!
- $M_{z}$ is a measurement in $Z$ basis resuliting in: $ \langle \psi_{UCCSD}| U_{i}^{\dagger} M_{z}^{\dagger} M_{z} U_{i} |\psi_{UCCSD}\rangle$ (**important!**)


## what are the $U_{i}$ operations?

- These are **single** qubit transforms depending on Pauli required to be measured

1. $P=Z$ THEN $U_{i} = \mathbb{1}$
2. $P=X$ THEN $U_{i} = R_{Y}\big(-\frac{\pi}{2} \big)$
3. $P=Y$ THEN $U_{i} = R_{X}\big(+\frac{\pi}{2} \big)$

EXAMPLE 1

if $P=Z$ and state is $|0/1\rangle$:

$ |0/1\rangle$ == measure ==> $ M_{z}\mathbb{1}|0/1\rangle$ ==>  $Z|0/1\rangle$


if $P=X$ and state is $|\pm\rangle$:

$ |\pm\rangle$ == measure ==> $ M_{z} R_{Y}\big(-\frac{\pi}{2} \big)|\pm\rangle$ ==>  $M_{z}|0/1\rangle$




$$X|\pm\rangle = ZH|\pm\rangle = Z|0/1\rangle$$ 

$$Y|i\pm \rangle = ZR_{x}\big(-\frac{\pi}{2} \big)|i\pm\rangle = Z|0/1\rangle$$ 

- note $Z$ here is a measurement in Z basis!

In [7]:
# checking in python:

### measuring Y ###
theta = np.pi/2
phi = np.pi/2 # 3*np.pi/2

state =  np.array([[np.cos(theta/2)],
                   [np.exp(1j*phi)*np.sin(theta/2)]])

state = np.around(state,5)

print('state = ', state)
print('')
print(np.dot(cirq.rx(np.pi/2)._unitary_(),state))


print('')
print('###')
print('')

### measuring X ###
theta = np.pi/2 #3*np.pi/2
phi = 0 

state =  np.array([[np.cos(theta/2)],
                   [np.exp(1j*phi)*np.sin(theta/2)]])

state = np.around(state,5)

print('state = ', state)
print('')
print(np.dot(cirq.ry(-np.pi/2)._unitary_(),state))


state =  [[0.70711+0.j     ]
 [0.     +0.70711j]]

[[1.00000455+0.j]
 [0.        +0.j]]

###

state =  [[0.70711+0.j]
 [0.70711+0.j]]

[[1.00000455+0.j]
 [0.        +0.j]]


overall we are doing

$$ E_{i} =  P_{i} |\psi_{UCCSD}\rangle = M_{z} U_{i}|\psi_{UCCSD}\rangle = M_{z}|\psi_{P_{i}}\rangle = \langle\psi_{P_{i}}|M_{z}^{\dagger} M_{z}|\psi_{P_{i}}\rangle $$ 

## Change_PauliWord_measurement_to_Z_basis

In [14]:
from quchem.Qcircuit.Hamiltonian_term_measurement_functions import Change_PauliWord_measurement_to_Z_basis

In [9]:
x = QubitOperator('X0 Z2 Y3', 0.25j)
full_P_M_Z_circ_obj = Change_PauliWord_measurement_to_Z_basis(x)
# print(cirq.Circuit((full_P_M_Z_circ_obj(*cirq.LineQubit.range(full_P_M_Z_circ_obj.num_qubits())))))
# print('####')
print(
    cirq.Circuit(cirq.decompose_once((full_P_M_Z_circ_obj(*cirq.LineQubit.range(full_P_M_Z_circ_obj.num_qubits()))))))

0: ───Ry(-0.5π)───

3: ───Rx(0.5π)────


## Measure_PauliWord

In [10]:
from quchem.Qcircuit.Hamiltonian_term_measurement_functions import Measure_PauliWord

In [11]:
x = QubitOperator('X0 Z2 Y3', 0.25j)
measure_circ_obj = Measure_PauliWord(x)
# print(cirq.Circuit((measure_circ_obj(*cirq.LineQubit.range(measure_circ_obj.num_qubits())))))
# print('####')
print(
    cirq.Circuit(cirq.decompose_once((measure_circ_obj(*cirq.LineQubit.range(measure_circ_obj.num_qubits()))))))

0: ───M───
      │
2: ───M───
      │
3: ───M───


## change_pauliword_to_Z_basis_then_measure

In [12]:
from quchem.Qcircuit.Hamiltonian_term_measurement_functions import change_pauliword_to_Z_basis_then_measure

In [13]:
x = QubitOperator('X0 Z2 Y3', 0.25j)
change_then_measure_circ_obj = change_pauliword_to_Z_basis_then_measure(x)
# print(cirq.Circuit((change_then_measure_circ_obj(*cirq.LineQubit.range(change_then_measure_circ_obj.num_qubits())))))
# print('####')
print(
    cirq.Circuit(cirq.decompose_once((change_then_measure_circ_obj(*cirq.LineQubit.range(change_then_measure_circ_obj.num_qubits()))))))

0: ───Ry(-0.5π)───M───
                  │
2: ───────────────M───
                  │
3: ───Rx(0.5π)────M───
